In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories2 import *
from projects.creek import test

In [2]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [3]:
version = 2

In [4]:
projects = [sudoku,seeknumbers,minotaur,creek]

In [5]:
for i in range(len(projects)):
    copy_problem = dict(projects[i])
    example_1 = projects[i-1]
    example_2 = projects[i-2]
    story = copy_problem["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    print(f'{story=}, {story_1=}, {story_2=}.')
    generation_rules = produce_generation_rules(example_1,example_2,copy_problem)
    definition_rules = produce_definition_rules(example_1,example_2,copy_problem)
    asp_code = [generation_rules,definition_rules]
    answer = '\n'.join(asp_code)
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    with open(name_of_file,"w") as file:
        file.write(answer)
    print("="*70)    

story='sudoku', story_1='creek', story_2='minotaur'.
story='seeknumbers', story_1='sudoku', story_2='creek'.
story='minotaur', story_1='seeknumbers', story_2='sudoku'.
story='creek', story_1='minotaur', story_2='seeknumbers'.


In [6]:
def asp_try(asp_file, instance):
    solution_lines = []
    task = ""
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:       
            with ctl.solve(yield_=True, async_=True) as handle: 
                handle.wait(10)
                handle.cancel()
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    #for m in models:
                    #    print(m.symbols(True,True,True))
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [7]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp")
                }

In [8]:
for i in range(len(projects)):
    query = dict(projects[i])
    story = query["story"]
    story_1 = projects[i-1]["story"]
    story_2 = projects[i-2]["story"]
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    print("="*40 + name_of_file + "="*40)
    with open(name_of_file,"r") as f:
        print("Length of File:" + str(len(f.readlines())))
    instance = instances_dict[story]
    success, models = asp_try(name_of_file,instance)
    print("Success: " + str(success))
    if models:
        for m in models:
            print(m.symbols(True,True,True))

========================================generated_solutions\sudoku_from_creek_and_minotaur_v2.lp========================================
Length of File:28
Parsing or Safety Issues
Success: 0
========================================generated_solutions\seeknumbers_from_sudoku_and_creek_v2.lp========================================
Length of File:55
Parsing or Safety Issues
Success: 0
========================================generated_solutions\minotaur_from_seeknumbers_and_sudoku_v2.lp========================================
Length of File:47
Parsing or Safety Issues
Success: 0
========================================generated_solutions\creek_from_minotaur_and_seeknumbers_v2.lp========================================
Length of File:53
Parsing or Safety Issues
Success: 0
